# Advanced Lane Detection

## (using Deep Learning)




### Import TensorFlow and Keras libraries

We will start by importing all the necessary libraries for Keras and TensorFlow. Apart from these libraries, we will also import some other libraries such as NumPy, Glob, cv2, csv, and MatPlotLib.


In [ ]:
import os
import csv
import cv2
import numpy as np
import keras
import tensorflow as tf
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import random
import math

from scipy import ndimage

from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Cropping2D
from keras.layers import Dropout

import sklearn
from sklearn.model_selection import train_test_split

import GPUtil as GPU

## GPU information

Here, we will print some information related to GPU memory capacity. The user can accordingly make modifications to the code in case of more/less capacity.

In [ ]:
import GPUtil as GPU
GPUs = GPU.getGPUs()
gpu = GPUs[0]
print("Total Available GPU memory: {} MB".format(gpu.memoryTotal))
print("Used GPU memory: {} MB".format(gpu.memoryUsed))
print("Total Free GPU memory: {} MB".format(gpu.memoryFree))

## Check library versions

Here, we print the versions of tensorflow, keras and OpenCV versions to compare compatibility.

In [ ]:
# printing installed tensorflow/keras/OpenCV versions
print("Current OpenCV version:", cv2.__version__)
print("Using TensorFlow version:", tf.__version__)
print("Using Keras version:", keras.__version__)

## Data Augmentation

We use this cell to activate data augmentation. This step is highly advised to correct for any undesired bias in the model. We create a bool parameter. If set to True, we will perform data augmentation.

(Note: This step may take a significant amount of time depending on the size of the original data).

In [ ]:
# setting data augmentation activation to True/False
augment = True


## Dataset building and Augmentation

Here, we load the data from the *driving_log.csv* file and if required, augment the data by flipping the images in the opposite direction.

In [ ]:
# 1. Start of dataset building
## Read the csv file containing saved model data

lines = []
with open('./data/model_data.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

# empty list to save images        
binary_images = []

# lists to save polynomial measurements for left and right lanes

left_measurements_a = []
left_measurements_b = []
left_measurements_c = []
right_measurements_a = []
right_measurements_b = []
right_measurements_c = []

# directory for accessing binary images
directory = '../data_for_lane_detection/binary_images/'
directory = '../data_for_lane_detection/lanelines_images/'
directory = '../data_for_lane_detection/transformed_images/'



if (augment):
    print("Performing data augmentation by flipping the images ...\n")

for line in lines:

    # reading filesnames for binary images and lanelines images
    fname_binary_img = directory + line[0].split('/')[-1]
    fname_lanelines_img = directory + line[1].split('/')[-1]
    fname_transformed_img = directory + line[2].split('/')[-1]

    # reading images using opencv
    bin_img = mpimg.imread(fname_binary_img)
    laneline_img = mpimg.imread(fname_lanelines_img)
    
    # adding images to the lists
    images.append(fname_binary_img)
    images.append(fname_lanelines_img)
    
    # adding steering angles for center. left and right images

    left_measurements_a.append(float(line[3]))
    left_measurements_b.append(float(line[4]))
    left_measurements_c.append(float(line[5]))
    right_measurements_a.append(float(line[6]))
    right_measurements_b.append(float(line[7]))
    right_measurements_c.append(float(line[8]))

    # only if data augmentation is required
    if (augment):
        
        # this is where I perform data augmentation for images and angles
        # binary_img_flipped = np.fliplr(fname_binary_img)
        # images.append(binary_img_flipped)
        

        # laneline_img_flipped = np.fliplr(fname_laneline_img)
        # images.append(lanelines_img_flipped)


        transformed_img_flipped = np.fliplr(fname_transformed_img)
        images.append(transformed_img_flipped)
        
        # flip the measurements
        left_measurements_a_flip = -(float(line[3]))
        left_measurements_b_flip = -(float(line[4]))
        left_measurements_c_flip = -(float(line[5]))
        right_measurements_a_flip = -(float(line[6]))
        right_measurements_b_flip = -(float(line[7]))
        right_measurements_c_flip = -(float(line[8]))
        
        # add the flipped measurements to the current measurements lists
        left_measurements_a.append(left_measurements_a_flip)
        left_measurements_b.append(left_measurements_b_flip)
        left_measurements_c.append(left_measurements_c_flip)
        right_measurements_a.append(right_measurements_a_flip)
        right_measurements_b.append(right_measurements_b_flip)
        right_measurements_c.append(right_measurements_c_flip)


# define arrays for datasets

X_train = np.array(images)

left_measurements_a = np.array(left_measurements_a)
left_measurements_b = np.array(left_measurements_b)
left_measurements_c = np.array(left_measurements_c)
right_measurements_a = np.array(right_measurements_a)
right_measurements = np.array(right_measurements_b)
right_measurements = np.array(right_measurements_c)


## Display sample flipped images

This is the procedure I used to augment the data. The image was flipped from left to right. Here is a sample flipped image.

In [ ]:
# select a random image from the dataset

idx = random.randint(0, len(X_train))
img = X_train[idx]
flipped_img = np.fliplr(img)

left_a = left_measurements_a[idx]
left_b = left_measurements_b[idx]
left_c = left_measurements_c[idx]
right_a = right_measurements_a[idx]
right_b = right_measurements_b[idx]
right_c = right_measurements_c[idx]

# flip the lane polynomial coefficients
flipped_left_a = -float(left_a)
flipped_left_b = -float(left_b)
flipped_left_c = -float(left_c)
flipped_right_a = -float(left_a)
flipped_right_b = -float(left_b)
flipped_right_c = -float(left_c)


plt.figure()
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(40, 20))

ax1.imshow(img)
ax1.set_title('Original Image' , fontsize=30)
ax2.imshow(flipped_img)
ax2.set_title('Flipped Image', fontsize=30)

plt.axis('off')
plt.show()


### Print Dataset information

We will print information regarding the dataset here. We will also print information after data augmentation. For each laneline, we have a set of three polynomial coefficients.

In [ ]:
# function to print the dataset information
def print_dataset_info():

    if (augment):
        # print out array shapes and details
        print("Original Dataset information ---\n")
        # print("Image shape: ", X_train[0][1])
        print("Measurement Sets:", 0.5*len(left_measurements_a))
        print("Total images:", 0.5*X_train.shape[0])
        print()

        print("Augmented Dataset information ---\n")
        # print("Image shape: ", X_train[0][1])
        print("Measurement sets after augmentation:", len(left_measurements_a))
        print("Total images after augmentation:", X_train.shape[0])
        print()
    else:
        # print out array shapes and details
        print("Original Dataset information ---\n")
        # print("Image shape: ", X_train[0][1])
        print("Measurement Sets:", 0.5*len(left_measurements_a))
        print("Total images:", 0.5*X_train.shape[0])

### Display a sample image

Here, we display a random sample image along with the drawn detected lanelines. We also print some characteristics of the lanelines polynomials.

In [ ]:
idx = random.randint(0, len(X_train))
sample_img = X_train[idx]
print(sample_img.shape)
plt.imshow(sample_img)
plt.axis('off')
plt.title("Measurement Sets - " + str(left_measurements_a[idx]), fontsize=15)
plt.show()

## Network Architecture

Now, we will define our Convolutional Neural Network here comtaining 6 convolutional layers, 4 fully connected layers and some dropout and lambda layers in between. We will use a dropout layer after the 6th convolutional layers and set the number of epochs to 10.


In [ ]:

# 2. Start of Network Architecture
# NVIDIA network architecture
model = Sequential()

# image preprocessing - normalizing the pixel values and cropping the image
model.add(Lambda(lambda x:(x/127.5)-1.0, input_shape=(160,320,3)))
# cropping top 50 pixels and 20 pixels from the bottom
model.add(Cropping2D(cropping=((50,20),(0,0)), input_shape=(160,320,3)))

# 1st Convolutional layer
model.add(Conv2D(24, (5, 5), subsample = (2,2), activation="relu"))
# 2nd Convolutional layer
model.add(Conv2D(36, (5, 5), subsample = (2,2), activation="relu"))
# 3rd Convolutional layer
model.add(Conv2D(48, (5, 5), subsample = (2,2), activation="relu"))
# 4th Convolutional layer
model.add(Conv2D(64, (3, 3), activation="relu"))
# 5th Convolutional layer
model.add(Conv2D(64, (3, 3), activation="relu"))
# 6th Convolutional layer
model.add(Conv2D(64, (3, 3), activation="relu"))

model.add(Dropout(0.5))

# flatten layers into a vector
model.add(Flatten())

# four fully connected layers
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='relu'))

# final layer to decide the 6 laneline polynomial coefficients
model.add(Dense(6))

model.compile(loss='mse', optimizer ='adam')

# fit model with a validation set of 20%
history_object = model.fit(X_train, steering_angles, validation_split=0.2, shuffle=True, epochs=10)

model.save('model.h5')

## Model Summary

Print the model summary.

In [ ]:
model.summary()

## Print Loss

Print the training loss. We will also plot a graph of training loss (mean squared error loss) vs 

In [ ]:
print(history_object.history.keys())

plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.ylim(0,0.1)
plt.legend(['training set', 'validation set'], loc='upper right')

plt.show()
